In [1]:
import pandas as pd
import numpy as np
import math
from math import sqrt
from math import exp
from math import pow
import array as arr

In [2]:
df = pd.read_csv('indexprice.csv')

In [3]:
columns_names=df.columns.tolist()
print("Columns names:")
print(columns_names)

Columns names:
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


In [4]:
df.shape

(248, 7)

In [5]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,17625.699219,200500
1,2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,17805.250000,247400
2,2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,17925.250000,251500
3,2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,17745.900391,236500
4,2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,17812.699219,239300


In [6]:
df["ror"]= ((df["Close"]/df["Open"]) -1) 

In [7]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ror
0,2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,17625.699219,200500,0.013720
1,2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,17805.250000,247400,0.007005
2,2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,17925.250000,251500,0.005901
3,2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,17745.900391,236500,-0.001272
4,2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,17812.699219,239300,0.000848


In [8]:
vol = df['ror']
vol.head()

0    0.013720
1    0.007005
2    0.005901
3   -0.001272
4    0.000848
Name: ror, dtype: float64

In [9]:
volatility = vol.agg([np.std]) *sqrt(252)
print(volatility)

std    0.130455
Name: ror, dtype: float64


In [10]:
std_dev  = vol.agg([np.std])
print(std_dev)

std    0.008218
Name: ror, dtype: float64


In [11]:
def rnp(N):
    dt = 200/(365*N)
    U = exp(volatility * sqrt(dt))
    D = 1/U
    r = 0.06
    R = exp(r*dt)
    alpha = (R-D)/(U-D)
    return alpha 

In [12]:
import yfinance as yf

nifty50 = yf.Ticker("^NSEI")

df = nifty50.history(period="1s")
S = df["Close"]
print(S)

Date
2023-03-03 00:00:00+05:30    17594.349609
Name: Close, dtype: float64


In [13]:
def bopm(N):
    dt = 200/(365*N)
    U = exp(std_dev * sqrt(dt))
    D = 1/U
    matrix = np.zeros((N+1,N+1))
    for j in range(N+1):
        for i in range(j+1):
            matrix[i,j] = S*pow(D,i) * pow(U,j-i)
    return matrix

In [14]:
x = bopm(4)
print(x)

[[17594.34960938 17647.94573123 17701.70511824 17755.62826775
  17809.71567861]
 [    0.         17540.91625685 17594.34960938 17647.94573123
  17701.70511824]
 [    0.             0.         17487.64517933 17540.91625685
  17594.34960938]
 [    0.             0.             0.         17434.53588399
  17487.64517933]
 [    0.             0.             0.             0.
  17381.58787951]]


In [15]:
y = bopm(5)
print(y)

[[17594.34960938 17642.2797384  17690.34043759 17738.53206266
  17786.85497025 17835.30951802]
 [    0.         17546.5496958  17594.34960938 17642.2797384
  17690.34043759 17738.53206266]
 [    0.             0.         17498.87964389 17546.5496958
  17594.34960937 17642.2797384 ]
 [    0.             0.             0.         17451.33910086
  17498.87964389 17546.5496958 ]
 [    0.             0.             0.             0.
  17403.92771486 17451.33910086]
 [    0.             0.             0.             0.
      0.         17356.64513498]]


In [16]:
z = bopm(10)
print(z)

[[17594.34960938 17628.22782349 17662.17127068 17696.18007654
  17730.25436693 17764.39426793 17798.59990588 17832.87140736
  17867.20889919 17901.61250843 17936.08236239]
 [    0.         17560.53650296 17594.34960938 17628.22782349
  17662.17127068 17696.18007654 17730.25436693 17764.39426793
  17798.59990588 17832.87140736 17867.20889919]
 [    0.             0.         17526.78837911 17560.53650296
  17594.34960938 17628.22782349 17662.17127068 17696.18007654
  17730.25436693 17764.39426793 17798.59990588]
 [    0.             0.             0.         17493.10511296
  17526.78837911 17560.53650296 17594.34960938 17628.22782349
  17662.17127068 17696.18007654 17730.25436693]
 [    0.             0.             0.             0.
  17459.48657986 17493.10511296 17526.78837911 17560.53650296
  17594.34960938 17628.22782349 17662.17127068]
 [    0.             0.             0.             0.
      0.         17425.9326554  17459.48657986 17493.10511296
  17526.78837911 17560.53650296 

In [17]:
#Exercise Price
E = 17500

In [18]:
def call(N):
    dt = 200/(365*N)
    R = exp(0.06*dt)
    pro = rnp(N)
    matrix = bopm(N)
    a = [0] * (N+1)
    for i in range(N):
        a[i] = matrix[i,N]
    for j in range(N+1):
        if(j==0):
            p = [None] * (N-j+1)
            for k in range(N+1):
                p[k] = max(0,a[k]-E)
        else:
            q=p
            for k in range(N+1-j):
                x = p[k]*pro
                y = p[k+1]*(1-pro)
                q[k] = (x+y)/R
    op = q[0]
    return op

In [19]:
def put(N):
    dt = 200/(365*N)
    R = exp(0.06*dt)
    pro = rnp(N)
    matrix = bopm(N)
    a = [0] * (N+1)
    for i in range(N):
        a[i] = matrix[i,N]
    for j in range(N+1):
        if(j==0):
            p = [None] * (N-j+1)
            for k in range(N+1):
                p[k] = max(0,E-a[k])
        else:
            q=p
            for k in range(N+1-j):
                x = p[k]*pro
                y = p[k+1]*(1-pro)
                q[k] = (x+y)/R
    op = q[0]
    return op

In [20]:
#For N = 4
call4 = call(4)
print("Call option price for N = 4 is - ",call4)
put4 = put(4)
print("Put option price for N = 4 is - ",put4)

Call option price for N = 4 is -  127.95141895725948
Put option price for N = 4 is -  563.1719313143639


In [21]:
#For N = 5
call5 = call(5)
print("Call option price for N = 5 is - ",call5)
put5 = put(5)
print("Put option price for N = 5 is - ",put5)

Call option price for N = 5 is -  128.8906517393389
Put option price for N = 5 is -  266.80784257211303


In [22]:
#For N = 10
call10 = call(10)
print("Call option price for N = 10 is - ",call10)
put10 = put(10)
print("Put option price for N = 10 is - ",put10)

Call option price for N = 10 is -  128.33366544982462
Put option price for N = 10 is -  12.558486399362902
